# Hello Window
To begin with, we need to make sure we have all the required modules installed.

In [ ]:
!python -m pip install -r requirements.txt

Now we can start by creating our first OpenGL capable application.

Let's see if we can get GLFW up and running. First, create a `.py` file and add the following imports to the top of your newly created file.

In [1]:
from OpenGL.GL import *
from glfw.GLFW import *
from glfw import _GLFWwindow as GLFWwindow

Next, we create the `main` function where we will instantiate the GLFW window:

In [2]:
def main() -> int:
    glfwInit()
    glfwWindowHint(GLFW_CONTEXT_VERSION_MAJOR, 3)
    glfwWindowHint(GLFW_CONTEXT_VERSION_MINOR, 3)
    glfwWindowHint(GLFW_OPENGL_PROFILE, GLFW_OPENGL_CORE_PROFILE)
    # glfwWindowHint(GLFW_OPENGL_FORWARD_COMPAT, GL_TRUE)

    # See the next step
    return step2()

In the main function we first initialize GLFW with `glfwInit`, after which we can configure GLFW using `glfwWindowHint`. The first argument of `glfwWindowHint` tells us what option we want to configure, where we can select the option from a large enum of possible options prefixed with `GLFW_`. The second argument is an integer that sets the value of our option. A list of all the possible options and its corresponding values can be found at [GLFW's window handling](http://www.glfw.org/docs/latest/window.html#window_hints) documentation.

Since the focus of this book is on OpenGL version 3.3 we'd like to tell GLFW that 3.3 is the OpenGL version we want to use. This way GLFW can make the proper arrangements when creating the OpenGL context. This ensures that when a user does not have the proper OpenGL version GLFW fails to run. We set the major and minor version both to 3. We also tell GLFW we want to explicitly use the core-profile. Telling GLFW we want to use the core-profile means we'll get access to a smaller subset of OpenGL features without backwards-compatible features we no longer need. Note that on Mac OS X you need to add `glfwWindowHint(GLFW_OPENGL_FORWARD_COMPAT, GL_TRUE)` to your initialization code for it to work.

Next we're required to create a window object. This window object holds all the windowing data and is required by most of GLFW's other functions.

In [3]:
def step2() -> int:
    window: GLFWwindow = glfwCreateWindow(800, 600, "LearnOpenGL", None, None)

    if (window == None):
        print("Failed to create GLFW window")
        glfwTerminate()
        return -1

    glfwMakeContextCurrent(window)

    # See the next step
    return step3(window)

The `glfwCreateWindow` function requires the window width and height as its first two arguments respectively. The third argument allows us to create a name for the window; for now we call it `"LearnOpenGL"` but you're allowed to name it however you like. We can ignore the last 2 parameters. The function returns a `GLFWwindow` object that we'll later need for other GLFW operations. After that we tell GLFW to make the context of our window the main context on the current thread.

## Viewport
Before we can start rendering we have to do one last thing. We have to tell OpenGL the size of the rendering window so OpenGL knows how we want to display the data and coordinates with respect to the window. We can set those *dimensions* via the `glViewport` function:

In [4]:
def optional_step():
    glViewport(0, 0, 800, 600)

The first two parameters of `glViewport` set the location of the lower left corner of the window. The third and fourth parameter set the width and height of the rendering window in pixels, which we set equal to GLFW's window size.

We could actually set the viewport dimensions at values smaller than GLFW's dimensions; then all the OpenGL rendering would be displayed in a smaller window and we could for example display other elements outside the OpenGL viewport.

Behind the scenes OpenGL uses the data specified via glViewport to transform the 2D coordinates it processed to coordinates on your screen. For example, a processed point of location **`(-0.5,0.5)`** would (as its final transformation) be mapped to **`(200,450)`** in screen coordinates. Note that processed coordinates in OpenGL are between `-1` and `1` so we effectively map from the range `(-1 to 1)` to `(0, 800)` and `(0, 600)`.</p>

However, the moment a user resizes the window the viewport should be adjusted as well. We can register a callback function on the window that gets called each time the window is resized. This resize callback function has the following prototype:

In [5]:
def framebuffer_size_callback(window: GLFWwindow, width: int, height: int) -> None:
    pass

The framebuffer size function takes a `GLFWwindow` as its first argument and two integers indicating the new window dimensions. Whenever the window changes in size, GLFW calls this function and fills in the proper arguments for you to process.

In [6]:
def framebuffer_size_callback(window: GLFWwindow, width: int, height: int) -> None:
    glViewport(0, 0, width, height)

We do have to tell GLFW we want to call this function on every window resize by registering it:

In [7]:
def step3(window: GLFWwindow) -> int:
    glfwSetFramebufferSizeCallback(window, framebuffer_size_callback)

    return step4(window)

When the window is first displayed `framebuffer_size_callback` gets called as well with the resulting window dimensions. For retina displays `width` and `height` will end up significantly higher than the original input values.

There are many callbacks functions we can set to register our own functions. For example, we can make a callback function to process joystick input changes, process error messages etc. We register the callback functions after we've created the window and before the render loop is initiated.

## Ready your engines
We don't want the application to draw a single image and then immediately quit and close the window. We want the application to keep drawing images and handling user input until the program has been explicitly told to stop. For this reason we have to create a while loop, that we now call the `render loop`, that keeps on running until we tell GLFW to stop. The following code shows a very simple render loop:

In [8]:
def step4(window: GLFWwindow) -> int:
    while (not glfwWindowShouldClose(window)):
        glfwSwapBuffers(window)
        glfwPollEvents()
    
    return step5()

The `glfwWindowShouldClose` function checks at the start of each loop iteration if GLFW has been instructed to close. If so, the function returns `True` and the render loop stops running, after which we can close the application.
The `glfwPollEvents` function checks if any events are triggered (like keyboard input or mouse movement events), updates the window state, and calls the corresponding functions (which we can register via callback methods). The `glfwSwapBuffers` will swap the color buffer (a large 2D buffer that contains color values for each pixel in GLFW's window) that is used to render to during this render iteration and show it as output to the screen.

### Double buffer
When an application draws in a single buffer the resulting image may display flickering issues. This is because the resulting output image is not drawn in an instant, but drawn pixel by pixel and usually from left to right and top to bottom. Because this image is not displayed at an instant to the user while still being rendered to, the result may contain artifacts. To circumvent these issues, windowing applications apply a double buffer for rendering. The **front** buffer contains the final output image that is shown at the screen, while all the rendering commands draw to the **back** buffer. As soon as all the rendering commands are finished we **swap** the back buffer to the front buffer so the image can be displayed without still being rendered to, removing all the aforementioned artifacts.

## One last thing
As soon as we exit the render loop we would like to properly clean/delete all of GLFW's resources that were allocated. We can do this via the `glfwTerminate` function that we call at the end of the `main` function.

In [9]:
def step5() -> int:
    glfwTerminate()
    return 0

This will clean up all the resources and properly exit the application. Now try to run your application:

In [10]:
main()

0

You should see the following output:

![Hello Window](https://learnopengl.com/img/getting-started/hellowindow.png)

If it's a very dull and boring black image, you did things right! If you didn't get the right image or you're confused as to how everything fits together, check the full source code [here](hello_window.py) (and if it started flashing different colors, keep reading).

## Input
We also want to have some form of input control in GLFW and we can achieve this with several of GLFW's input functions. We'll be using GLFW's `glfwGetKey` function that takes the window as input together with a key. The function returns whether this key is currently being pressed. We're creating a `processInput` function to keep all input code organized:

In [11]:
def processInput(window: GLFWwindow) -> None:
    if(glfwGetKey(window, GLFW_KEY_ESCAPE) == GLFW_PRESS):
        glfwSetWindowShouldClose(window, True)

Here we check whether the user has pressed the escape key (if it's not pressed, `glfwGetKey` returns `GLFW_RELEASE`). If the user did press the escape key, we close GLFW by setting its `WindowShouldClose` property to `True` using `glfwSetWindowShouldClose`. The next condition check of the main while loop will then fail and the application closes.

We then call `processInput` every iteration of the render loop:

In [12]:
def step4(window: GLFWwindow) -> int:
    while (not glfwWindowShouldClose(window)):
        processInput(window)
        
        glfwSwapBuffers(window)
        glfwPollEvents()
    
    return step5()

This gives us an easy way to check for specific key presses and react accordingly every frame. An iteration of the render loop is more commonly called a `frame`.

## Rendering
We want to place all the rendering commands in the render loop, since we want to execute all the rendering commands each iteration or frame of the loop. This would look a bit like this:

In [13]:
def step4(window: GLFWwindow) -> int:
    # render loop
    while (not glfwWindowShouldClose(window)):
        # input
        processInput(window)

        # rendering commands here
        step4a()
        
        # check and call events and swap the buffers
        glfwSwapBuffers(window)
        glfwPollEvents()
    
    return step5()

Just to test if things actually work we want to clear the screen with a color of our choice. At the start of frame we want to clear the screen. Otherwise we would still see the results from the previous frame (this could be the effect you're looking for, but usually you don't). We can clear the screen's color buffer using `glClear` where we pass in buffer bits to specify which buffer we would like to clear. The possible bits we can set are `GL_COLOR_BUFFER_BIT`, `GL_DEPTH_BUFFER_BIT` and `GL_STENCIL_BUFFER_BIT`. Right now we only care about the color values so we only clear the color buffer.

In [14]:
def step4a() -> None:
    glClearColor(0.2, 0.3, 0.3, 1.0)
    glClear(GL_COLOR_BUFFER_BIT)

Note that we also specify the color to clear the screen with using `glClearColor`. Whenever we call `glClear` and clear the color buffer, the entire color buffer will be filled with the color as configured by `glClearColor`. This will result in a dark green-blueish color.

Now to re-run the application:

In [15]:
main()

0

![Hello Window 2](https://learnopengl.com/img/getting-started/hellowindow2.png)

Source: https://learnopengl.com/Getting-started/Hello-Window

Original Author: Joey De Vries

Modified by: Zuzu_Typ

Licensed under the terms of the CC BY-NC 4.0 license.

See https://learnopengl.com/About for more information.